In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import sympy as smp
from matplotlib import animation
from matplotlib.animation import PillowWriter

### Theroy

#### Torque

The net torque $ \tau$ on a body determines the rate of change of the body's angular momentum

$ \tau=\frac{d\vec{L}}{dt} $ ,     $ \vec{L}=I\vec{\omega}$

$ \tau=I\frac{d\vec{w}}{dt} $

$ \tau=I \ddot{\theta}$
where $I$ is moment of inertia of the body, given by $I=\sum{mr^2}$.

The moment of inertia of a rod of uniform quality is 
$I=\frac{1}{3}mL^2$, where $m$ and $L$ are the mass and length of rod.

#### Lagrange equations with non-conservation force

The conservative forces are involved in Lagrangian, and the non-conservative forces present in the form of generalized forces $Q$.

$\frac{d}{dt} \frac{\partial L}{\partial \dot{q_j}}- \frac{\partial L}{\partial q_j}=Q_j$, $\;\;\;\;\;\;$ $j=1,2,3,....s$

$Q_j=\sum^{n}_{i=1} \vec{F_i} \cdot \frac{\partial \vec{r_i}}{ \partial q_j} $

where $\vec{r_j}$ are Cartesian coordinates, $q_j$ are generalized coordinates.

### First, we consider the single pendulum.

Define symbolic variables:

In [27]:
tau, u, t, g, L, m, I = smp.symbols('tau u t g L m I')
theta, Lag, x1, y1,x2,y2, Q, Fx, Fy, LE = smp.symbols('theta Lag x1 y1 x2 y2 Q F_x F_y LE', cls=smp.Function)

In [28]:
theta = theta(t)
theta_d = smp.diff(theta, t)
theta_dd = smp.diff(theta_d, t)
theta_dd

Derivative(theta(t), (t, 2))

In [29]:
from IPython.display import Image
Image(url= "assets\single_pendulum.jpg")

The above result is $\ddot{\theta}=1.5*g*sin(\theta(t))/L + 3.0*u/(L^2*m)$

Subsutiting the above equation into the below equation

$\dot{\theta}_{new}=\dot{\theta}_{old}+\ddot{\theta}*dt$

Then we got,
$\dot{\theta}_{new}=\dot{\theta}_{old}+(1.5*g*sin(\theta_1(t))/L + 3.0*u/(L^2*m_1))*dt$

The new $\theta_{new}$ is given by
$\theta_{new}=\theta_{old}+\dot{\theta}_{new}*dt$

So far our observations $\dot{\theta}_{new}$ and $\theta_{new}$ can be calculated step by step.

In [30]:
I=smp.Rational(1,3)*m*L**2
# I=m*L**2
tau=smp.Rational(1,2)*m*g*L*smp.sin(theta)+u
the1_dd_sol=smp.solve(tau-I*theta_dd, theta_dd, simplify=True, rational=False)
the1_dd_sol

[3*g*sin(theta(t))/(2*L) + 3*u/(L**2*m)]

$\ddot{\theta}=3*g*sin(\theta)/(2*L) + 3*u/(L^2*m)$

Lagrangian method

In [6]:
#position of mass center
x1=L/2*smp.cos(theta)
y1=L/2*smp.sin(theta)
#force 
Fx=-u/L*smp.sin(theta)
Fy=u/L*smp.cos(theta)
#position of force acting on rod
x2=L*smp.cos(theta)
y2=L*smp.sin(theta)
Q=Fx*smp.diff(x2,theta)+Fy*smp.diff(y2,theta)
Q

u*sin(theta(t))**2 + u*cos(theta(t))**2

In [7]:
Lag=smp.Rational(1,2)*I*(smp.diff(theta, t)**2)-(m*g*y1)
Lag

L**2*m*Derivative(theta(t), t)**2/6 - L*g*m*sin(theta(t))/2

In [8]:
LE=smp.diff(smp.diff(Lag,theta_d), t)-smp.diff(Lag,theta)-Q
LE

L**2*m*Derivative(theta(t), (t, 2))/3 + L*g*m*cos(theta(t))/2 - u*sin(theta(t))**2 - u*cos(theta(t))**2

In [9]:
the2_dd_sol=smp.solve(LE, theta_dd, simplify=True, rational=False)
the2_dd_sol

[-3*g*cos(theta(t))/(2*L) + 3*u/(L**2*m)]

$\ddot{\theta}=-3*g*sin(\theta)/(2*L) + 3*u/(L^2*m)$

This is as same as the result of the previous method.

### Second, we consider the double pendulum.

In [10]:
Image(url= "assets\double_pendulum.jpg")

In [11]:
t, g, IC1, IC2, l1, l2, m1, m2 = smp.symbols('t g I_C1 I_C2 l_1 l_2 m_1 m_2')
the1, the2 = smp.symbols('theta_1 theta_2', cls=smp.Function)

In [12]:
IC1=smp.Rational(1,12)*m2*l1**2
IC2=smp.Rational(1,12)*m2*l2**2
the1 = the1(t)
the2 = the2(t)
the1_d = smp.diff(the1, t)
the2_d = smp.diff(the2, t)
the1_dd = smp.diff(the1_d, t)
the2_dd = smp.diff(the2_d, t)

In [13]:
x1,y1, x2, y2, x3,y3= smp.symbols('x_1 y_1 x_2 y_2 x_3 y_3')
#position of mass center of Pole_1 and Pole_2, and position r3=(x3,y3) where force acting on 

In [14]:
x1 = l1/2*smp.sin(the1)
y1 = -l1/2*smp.cos(the1)
x2 = l1*smp.sin(the1)+l2/2*smp.sin(the2)
y2 = -l1*smp.cos(the1)-l2/2*smp.cos(the2)
x3 = l1*smp.sin(the1)+l2*smp.sin(the2)
y3 = -l1*smp.cos(the1)-l2*smp.cos(the2)

In [15]:
T, U= smp.symbols('T U', cls=smp.Function)
#Kinetic energy and potential energy
T=smp.Rational(1,2)*(m1*(smp.diff(x1,t)**2+smp.diff(y1,t)**2)+m2*(smp.diff(x2,t)**2+smp.diff(y2,t)**2))+smp.Rational(1,2)*(IC1*the1_d**2+IC2*the2_d**2)
U=m1*g*y1+m2*g*y2
T=smp.simplify(T)
T

l_1**2*m_1*Derivative(theta_1(t), t)**2/8 + 13*l_1**2*m_2*Derivative(theta_1(t), t)**2/24 + l_1*l_2*m_2*cos(theta_1(t) - theta_2(t))*Derivative(theta_1(t), t)*Derivative(theta_2(t), t)/2 + l_2**2*m_2*Derivative(theta_2(t), t)**2/6

In [16]:
Lar2= smp.symbols('Lar2', cls=smp.Function)
Lar2=smp.simplify(T-U)
Lar2


g*l_1*m_1*cos(theta_1(t))/2 + g*m_2*(2*l_1*cos(theta_1(t)) + l_2*cos(theta_2(t)))/2 + l_1**2*m_1*Derivative(theta_1(t), t)**2/8 + 13*l_1**2*m_2*Derivative(theta_1(t), t)**2/24 + l_1*l_2*m_2*cos(theta_1(t) - theta_2(t))*Derivative(theta_1(t), t)*Derivative(theta_2(t), t)/2 + l_2**2*m_2*Derivative(theta_2(t), t)**2/6

Assuming the force is perpendicular to the line that crossing axel and the far end of Pole_2

In [17]:
Image(url= "assets\Angle.jpg")

In [18]:
# a is x, b is y
alpha, a, b, l3=smp.symbols('alpha a b l_3')
b=smp.pi+the1-the2
l3=smp.sqrt(l1**2+l2**2-2*l1*l2*smp.cos(b))
a=smp.asin(smp.sin(b)/l3*l2)
alpha=smp.simplify(the1+a)
alpha

theta_1(t) - asin(l_2*sin(theta_1(t) - theta_2(t))/sqrt(l_1**2 + 2*l_1*l_2*cos(theta_1(t) - theta_2(t)) + l_2**2))

In [19]:
F=smp.symbols('F')
Q1, Q2, Fx, Fy=smp.symbols('Q_1 Q_2 F_x F_y', cls=smp.Function)
Fx=F*smp.cos(alpha)
Fy=F*smp.sin(alpha)
Q1=Fx*smp.diff(x3,the1)+Fy*smp.diff(y3,the1)
Q2=Fx*smp.diff(x3,the2)+Fy*smp.diff(y3,the2)

Get Lagrange's equations

$$ \frac{d}{dt}\frac{\partial L}{\partial \dot{\theta_1}}-\frac{\partial L}{\partial \theta_1} = Q_1$$
$$\frac{d}{dt}\frac{\partial L}{\partial \dot{\theta_2}} -\frac{\partial L}{\partial \theta_2}= Q_2$$

In [20]:
LE1=smp.diff(smp.diff(Lar2,the1_d),t)-smp.diff(Lar2,the1)-Q1
LE2=smp.diff(smp.diff(Lar2,the2_d),t)-smp.diff(Lar2,the2)-Q2
sols= smp.solve([LE1, LE2], (the1_dd, the2_dd),simplify=True, rational=True)

Now we have 

* $\frac{d^2 \theta_1}{dt^2} = ...$
* $\frac{d^2 \theta_2}{dt^2} = ...$

In [21]:
sols[the1_dd]

(12*F*sin(theta_1(t) - asin(l_2*sin(theta_1(t) - theta_2(t))/sqrt(l_1**2 + 2*l_1*l_2*cos(theta_1(t) - theta_2(t)) + l_2**2)))*sin(theta_1(t)) - 18*F*sin(theta_1(t) - asin(l_2*sin(theta_1(t) - theta_2(t))/sqrt(l_1**2 + 2*l_1*l_2*cos(theta_1(t) - theta_2(t)) + l_2**2)))*sin(theta_2(t))*cos(theta_1(t) - theta_2(t)) - 18*F*cos(theta_1(t) - theta_2(t))*cos(theta_1(t) - asin(l_2*sin(theta_1(t) - theta_2(t))/sqrt(l_1**2 + 2*l_1*l_2*cos(theta_1(t) - theta_2(t)) + l_2**2)))*cos(theta_2(t)) + 12*F*cos(theta_1(t) - asin(l_2*sin(theta_1(t) - theta_2(t))/sqrt(l_1**2 + 2*l_1*l_2*cos(theta_1(t) - theta_2(t)) + l_2**2)))*cos(theta_1(t)) - 6*g*m_1*sin(theta_1(t)) - 12*g*m_2*sin(theta_1(t)) + 9*g*m_2*sin(theta_2(t))*cos(theta_1(t) - theta_2(t)) - 9*l_1*m_2*sin(2*theta_1(t) - 2*theta_2(t))*Derivative(theta_1(t), t)**2/2 - 6*l_2*m_2*sin(theta_1(t) - theta_2(t))*Derivative(theta_2(t), t)**2)/(l_1*(3*m_1 - 9*m_2*cos(theta_1(t) - theta_2(t))**2 + 13*m_2))

In [22]:
smp.simplify(sols[the2_dd])

3*(6*F*m_1*sin(theta_1(t) - asin(l_2*sin(theta_1(t) - theta_2(t))/sqrt(l_1**2 + 2*l_1*l_2*cos(theta_1(t) - theta_2(t)) + l_2**2)))*sin(theta_2(t)) + 6*F*m_1*cos(theta_1(t) - asin(l_2*sin(theta_1(t) - theta_2(t))/sqrt(l_1**2 + 2*l_1*l_2*cos(theta_1(t) - theta_2(t)) + l_2**2)))*cos(theta_2(t)) - 12*F*m_2*sin(theta_1(t) - asin(l_2*sin(theta_1(t) - theta_2(t))/sqrt(l_1**2 + 2*l_1*l_2*cos(theta_1(t) - theta_2(t)) + l_2**2)))*sin(theta_1(t))*cos(theta_1(t) - theta_2(t)) + 26*F*m_2*sin(theta_1(t) - asin(l_2*sin(theta_1(t) - theta_2(t))/sqrt(l_1**2 + 2*l_1*l_2*cos(theta_1(t) - theta_2(t)) + l_2**2)))*sin(theta_2(t)) - 12*F*m_2*cos(theta_1(t) - theta_2(t))*cos(theta_1(t) - asin(l_2*sin(theta_1(t) - theta_2(t))/sqrt(l_1**2 + 2*l_1*l_2*cos(theta_1(t) - theta_2(t)) + l_2**2)))*cos(theta_1(t)) + 26*F*m_2*cos(theta_1(t) - asin(l_2*sin(theta_1(t) - theta_2(t))/sqrt(l_1**2 + 2*l_1*l_2*cos(theta_1(t) - theta_2(t)) + l_2**2)))*cos(theta_2(t)) + 6*g*m_1*m_2*sin(theta_1(t))*cos(theta_1(t) - theta_2(t)) - 

In [23]:
sols[the1_dd].simplify

<bound method Basic.simplify of (12*F*sin(theta_1(t) - asin(l_2*sin(theta_1(t) - theta_2(t))/sqrt(l_1**2 + 2*l_1*l_2*cos(theta_1(t) - theta_2(t)) + l_2**2)))*sin(theta_1(t)) - 18*F*sin(theta_1(t) - asin(l_2*sin(theta_1(t) - theta_2(t))/sqrt(l_1**2 + 2*l_1*l_2*cos(theta_1(t) - theta_2(t)) + l_2**2)))*sin(theta_2(t))*cos(theta_1(t) - theta_2(t)) - 18*F*cos(theta_1(t) - theta_2(t))*cos(theta_1(t) - asin(l_2*sin(theta_1(t) - theta_2(t))/sqrt(l_1**2 + 2*l_1*l_2*cos(theta_1(t) - theta_2(t)) + l_2**2)))*cos(theta_2(t)) + 12*F*cos(theta_1(t) - asin(l_2*sin(theta_1(t) - theta_2(t))/sqrt(l_1**2 + 2*l_1*l_2*cos(theta_1(t) - theta_2(t)) + l_2**2)))*cos(theta_1(t)) - 6*g*m_1*sin(theta_1(t)) - 12*g*m_2*sin(theta_1(t)) + 9*g*m_2*sin(theta_2(t))*cos(theta_1(t) - theta_2(t)) - 9*l_1*m_2*sin(2*theta_1(t) - 2*theta_2(t))*Derivative(theta_1(t), t)**2/2 - 6*l_2*m_2*sin(theta_1(t) - theta_2(t))*Derivative(theta_2(t), t)**2)/(l_1*(3*m_1 - 9*m_2*cos(theta_1(t) - theta_2(t))**2 + 13*m_2))>

In [24]:
sols[the2_dd].simplify

<bound method Basic.simplify of 3*(6*F*m_1*sin(theta_1(t) - asin(l_2*sin(theta_1(t) - theta_2(t))/sqrt(l_1**2 + 2*l_1*l_2*cos(theta_1(t) - theta_2(t)) + l_2**2)))*sin(theta_2(t)) + 6*F*m_1*cos(theta_1(t) - asin(l_2*sin(theta_1(t) - theta_2(t))/sqrt(l_1**2 + 2*l_1*l_2*cos(theta_1(t) - theta_2(t)) + l_2**2)))*cos(theta_2(t)) - 12*F*m_2*sin(theta_1(t) - asin(l_2*sin(theta_1(t) - theta_2(t))/sqrt(l_1**2 + 2*l_1*l_2*cos(theta_1(t) - theta_2(t)) + l_2**2)))*sin(theta_1(t))*cos(theta_1(t) - theta_2(t)) + 26*F*m_2*sin(theta_1(t) - asin(l_2*sin(theta_1(t) - theta_2(t))/sqrt(l_1**2 + 2*l_1*l_2*cos(theta_1(t) - theta_2(t)) + l_2**2)))*sin(theta_2(t)) - 12*F*m_2*cos(theta_1(t) - theta_2(t))*cos(theta_1(t) - asin(l_2*sin(theta_1(t) - theta_2(t))/sqrt(l_1**2 + 2*l_1*l_2*cos(theta_1(t) - theta_2(t)) + l_2**2)))*cos(theta_1(t)) + 26*F*m_2*cos(theta_1(t) - asin(l_2*sin(theta_1(t) - theta_2(t))/sqrt(l_1**2 + 2*l_1*l_2*cos(theta_1(t) - theta_2(t)) + l_2**2)))*cos(theta_2(t)) + 6*g*m_1*m_2*sin(theta_1(t))

In [25]:
np.arcsin(4)

C:\Users\yqqPC\AppData\Local\Temp/ipykernel_5616/2518147911.py:1: RuntimeWarning: invalid value encountered in arcsin
  np.arcsin(4)


nan